# MCP tests

In [ ]:
# GPU-0 in devices, GPU-1 on the scheduler
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # note the GPU index as a string 

In [ ]:
import sys
from pathlib import Path

# this notebook is in notebooks — go up one level
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [ ]:
project_root

In [ ]:
from gdc_pipeline import execute_pipeline

In [ ]:
from langchain_core.tools import Tool

### wrap gdc_rag pipeline in a tool

In [ ]:
# init a dummy list
sys.argv = ['prog', '--input-file', 'dummy.csv']

In [ ]:
from gdc_pipeline import setup_args

In [ ]:
args = setup_args()

In [ ]:
args

In [ ]:
args.hf_token_path='.token'

In [ ]:
def wrapped_execute_pipeline(input_file: str):
    print(f'Tool received: {input_file}')
    return execute_pipeline(
        input_file=input_file,
        intent_model_path=args.intent_model_path,
        path_to_gdc_genes_mutations=args.path_to_gdc_genes_mutations_file,
        hf_token_path=args.hf_token_path
    )

In [ ]:
mcp_tool = Tool(
    name="run_pipeline_on_file",
    func=wrapped_execute_pipeline,
    description="Run the pipeline on a CSV file. Input is a file 'questions.csv'"
)

### define agent

In [ ]:
from langchain_core.language_models.fake import FakeListLLM

agent_llm = FakeListLLM(responses=[
    """Thought: I should use the tool
    Action: run_pipeline_on_file
    Action Input: questions.csv"""
])

### react style prompt template

In [ ]:
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
from langchain_core.prompts import PromptTemplate

template = """You are an assistant with access to a tool named `run_pipeline_on_file`. Your job is to run this tool on a file questions.csv.

{tools}

Thought: I should call the tool {tool_names}
Action: run_pipeline_on_file
Action Input: questions.csv

Begin!

Question: {input}
{agent_scratchpad}
""".strip()

prompt = PromptTemplate.from_template(
    template
)


### create agent and executor

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(
    llm=agent_llm,
    tools=[mcp_tool],
    prompt=prompt
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[mcp_tool],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1
)

### Run agent

In [ ]:
agent_executor.invoke(
    {'input': "Can you retrieve answers for the file questions.csv"}
)